## Pokročilé transformace dat v Pandas

- Pandas DataFrame je možné převádět z dlouhého formátu na široký (tj. převod řádků na sloupce) nebo ze širokého na dlouhý (tj. převod sloupců na řádky).
- V rámci těchto transformací lze u některých funkcí provádět agregaci hodnot.
- Při výběru konkrétní funkce je třeba zvážit:
    - Strukturu dat (chci pracovat s indexy / víceúrovňovými indexy, nebo zůstat u práce se sloupci).
    - Potřebu agregace dat (zda je agregace nutná či nikoliv).

Úvod do transformací v dokumentaci Pandas: [Pandas Reshaping Guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-stacking)

### Pivot

- **Kdy použít:** Když chcete přeformátovat data z dlouhého formátu do širokého bez agregace. Typické pro situace, kde je jasné mapování mezi indexem, sloupci a hodnotami a každá kombinace indexu a sloupce je unikátní.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako nový index.
  - `columns`: Sloupec nebo sloupce, které se stanou novými sloupci DataFrame.
  - `values`: Sloupec nebo sloupce, které se použijí pro vyplnění hodnot v novém DataFrame.

### Pivot Table

- **Kdy použít:** Pro přetváření dat z dlouhého formátu do širokého s možností agregace, vhodné pro situace, kde se mohou vyskytovat duplicitní kombinace indexu a sloupce.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako nový index.
  - `columns`: Sloupec nebo sloupce, které se stanou novými sloupci DataFrame.
  - `values`: Sloupec nebo sloupce, které se použijí pro vyplnění hodnot.
  - `aggfunc`: Funkce nebo seznam funkcí pro agregaci dat, např. 'sum', 'mean'.

### Stack

- **Kdy použít:** Když potřebujete převést sloupce na řádky, často pro přeformátování širokého formátu do dlouhého, zejména u víceúrovňových sloupců.
- **Důležité parametry:**
  - `level`: Úroveň nebo úrovně indexů sloupců, které chcete "stohovat". Výchozí je poslední úroveň.

### Unstack

- **Kdy použít:** Pro převod úrovní indexu řádků na sloupce, čímž se z dlouhých dat stávají široká. Užitečné pro práci s víceúrovňovými indexy.
- **Důležité parametry:**
  - `level`: Specifická úroveň indexu, kterou chcete převést na sloupce. Výchozí je poslední úroveň.

### Crosstab

- **Kdy použít:** Pro výpočet kontingenční tabulky, která počítá frekvenci různých kombinací ve dvou nebo více sloupcích. Ideální pro srovnávací studie a statistickou analýzu.
- **Důležité parametry:**
  - `index`: Sloupec nebo sloupce, které se použijí jako řádky kontingenční tabulky.
  - `columns`: Sloupec nebo sloupce, které se stanou sloupci kontingenční tabulky.
  - `values`: Volitelně, sloupec, jehož hodnoty budou agregovány.
  - `aggfunc`: Funkce pro agregaci, použije se, pokud je zadán `values`.

### Melt

- **Kdy použít:** Pro transformaci širokého formátu dat do dlouhého, když "rozpustíte" několik sloupců do dvou: jednoho pro názvy proměnných a druhého pro jejich hodnoty. Vhodné pro přípravu dat na další analýzu nebo vizualizaci.
- **Důležité parametry:**
  - `id_vars`: Sloupec nebo sloupce, které mají zůstat jako identifikátory; tyto sloupce nebudou transformovány.
  - `value_vars`: Volitelně, specifické sloupce, které mají být rozpuštěny. Pokud není zadáno, všechny ostatní sloupce nezahrnuté v `id_vars` budou rozpuštěny.
  - `var_name`: Název nového sloupce, který bude obsahovat názvy původních sloupců. Výchozí hodnota je 'variable'.
  - `value_name`: Název nového sloupce, který bude obsahovat hodnoty odpovídajících sloupců. Výchozí hodnota je 'value'.


| Funkce        | Přetváří dlouhá data na široká | Přetváří široká data na dlouhá | Podporuje agregaci | Pracuje s víceúrovňovými indexy | Umožňuje specifikovat hodnoty/agregace            |
|---------------|--------------------------------|--------------------------------|--------------------|----------------------------------|---------------------------------------------------|
| `pivot`       | Ano                            | Ne                             | Ne                 | Ne                               | Ano, pro hodnoty                                 |
| `pivot_table` | Ano                            | Ne                             | Ano                | Ano                              | Ano, s výběrem agregační funkce                  |
| `stack`       | Ne                             | Ano                            | Ne                 | Ano                              | Ne                                               |
| `unstack`     | Ne                             | Ano                             | Ne                 | Ano                              | Ne                                               |
| `crosstab`    | Ano                            | Ne                             | Ano                | Ne                               | Ano, s výběrem agregační funkce                  |
| `melt`        | Ne                             | Ano                            | Ne                 | Ne                               | Ne, ale umožňuje výběr proměnných a hodnot       |


In [2]:
import pandas as pd 

df = pd.read_csv('https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv',delimiter=';')
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year

df = df.loc[(df['NAME'].isin(['RUZYNE','MOSNOV'])) & (df['ROK'].isin([2010,2011,2012]))]

df = df.groupby(['ROK','NAME']).agg({'TMAX':'max'}).reset_index()




In [3]:
# Transformace za použití pivot, hodnoty vznikají z TMAX, index je ROK, sloupce NAME
df.pivot(index='ROK',columns='NAME',values='TMAX')



NAME  MOSNOV  RUZYNE
ROK                 
2010    35.9    34.6
2011    33.3    30.1
2012    34.2    37.4

In [4]:
# Transformace za použití pivot_table, hodnoty vznikají z TMAX, index je ROK, sloupce NAME
df.pivot_table(index='ROK',columns='NAME',values='TMAX',aggfunc='max')


NAME  MOSNOV  RUZYNE
ROK                 
2010    35.9    34.6
2011    33.3    30.1
2012    34.2    37.4

In [5]:
# Transformace za použití unstack
df_unstack = df.set_index(['ROK','NAME'])
print(df_unstack)
df_unstack = df_unstack.stack()
print(df_unstack)

print(df_unstack.unstack(level=1))

             TMAX
ROK  NAME        
2010 MOSNOV  35.9
     RUZYNE  34.6
2011 MOSNOV  33.3
     RUZYNE  30.1
2012 MOSNOV  34.2
     RUZYNE  37.4
ROK   NAME        
2010  MOSNOV  TMAX    35.9
      RUZYNE  TMAX    34.6
2011  MOSNOV  TMAX    33.3
      RUZYNE  TMAX    30.1
2012  MOSNOV  TMAX    34.2
      RUZYNE  TMAX    37.4
dtype: float64
NAME       MOSNOV  RUZYNE
ROK                      
2010 TMAX    35.9    34.6
2011 TMAX    33.3    30.1
2012 TMAX    34.2    37.4


In [8]:
# Transformace za použití crosstab
pd.crosstab(index=df['ROK'],columns=df['NAME'],values=df['TMAX'],aggfunc='max')


NAME  MOSNOV  RUZYNE
ROK                 
2010    35.9    34.6
2011    33.3    30.1
2012    34.2    37.4

In [13]:
# Transformace za použití melt, id_vars jsou ROK a NAME, value_vars je TMAX
df.melt(id_vars=['ROK','NAME'],value_vars=['TMAX'])


ROK    NAME variable  value
0  2010  MOSNOV     TMAX   35.9
1  2010  RUZYNE     TMAX   34.6
2  2011  MOSNOV     TMAX   33.3
3  2011  RUZYNE     TMAX   30.1
4  2012  MOSNOV     TMAX   34.2
5  2012  RUZYNE     TMAX   37.4

### Cvičení 1

1. Načtěte data se srážkami z github do dataframe df
2. Sloupce `DATE` a `NAME` dosaďte do indexu tak, aby zároveň nezůstaly jakou sloupec `df``
3. Ze sloupců odstraňte sloupec `STATION`
4. Sloupce `df` upravte tak, aby vznikla `pd.Series` s indexy s hodnotami původních slooupců `DATE` (již na indexu je),`NAME` (již na indexu je) a nově i hodnotami s názvy jednotlivých metrik.
Nápověda: Budete potřebovat metodu převádějící široká data na dlouhá a zároveň metodu, která umí pracovat s indexy (je to jedna jediná metoda)


### Cvičení 2

1. Načtěte data se srážkami z github do dataframe df
2. Index nechejte jak je (výchozí int), sloupce PRCP, SNWD,TMAX,TMIN upravte tak, ať názvy metrik jsou v jednom sloupci a hodnoty v druhém sloupci. Sloupec s metrikami pojmenujte `udaj`, sloupec s hodnotami `hodnota`.


### Cvičení 3

1. Načtěte data se srážkami z github do dataframe df
2. Převeďte sloupec `DATE` na typ datetime
3. Vytvořte sloupec `ROK` s rokem
4. Upravte dataframe tak, aby obsahoval sumu sloupce 'PRCP' pro každý `ROK` a `NAME`

### Cvičení 4

1. Načtěte data se srážkami z github do dataframe df
2. `df` omezte na sloupce `DATE`, `NAME`, `PRCP`
3. Upravte `df` tak, aby ve sloupcích byly názvy stanic (NAME), na řádcích `DATE` a v hodnotách `PRCP`

Nápověda: Zde není třeba nic agregovat, převádíme dlouhá data na široká